In [23]:
import os
import pandas as pd
import numpy as np
from tqdm import tqdm
from itertools import product
import random
from collections import Counter, defaultdict
import warnings
import logging
import subprocess
from multiprocessing.pool import ThreadPool
import joblib

# SCikitlearn modules :
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, RandomizedSearchCV, StratifiedKFold
from sklearn.metrics import classification_report , roc_auc_score, matthews_corrcoef
from sklearn.metrics import roc_auc_score

# Scipy modules : 
from scipy.stats import fisher_exact
from statistics import mean


PATH_WORK = "/home/conchae/prediction_depolymerase_tropism/prophage_prediction/depolymerase_decipher/ficheros_28032023/Seqbased_model"
path_metrics = f"{PATH_WORK}/metric_files"

### Make the metrics files : 

In [6]:
def load_data():
    df_info = pd.read_csv(f"{PATH_WORK}/TropiGATv2.final_df_v2.tsv", sep="\t", header=0)
    df_prophages = df_info.drop_duplicates(subset=["Phage"], keep="first")
    dico_prophage_info = {row["Phage"]: {"prophage_strain": row["prophage_id"], "ancestor": row["Infected_ancestor"]} for _, row in df_prophages.iterrows()}
    return df_info, dico_prophage_info

def get_filtered_prophages(prophage, df_info, dico_prophage_info):
    to_exclude = set()
    to_keep = {prophage}
    df_prophage_group = df_info[(df_info["prophage_id"] == dico_prophage_info[prophage]["prophage_strain"]) & 
                                (df_info["Infected_ancestor"] == dico_prophage_info[prophage]["ancestor"])]
    
    if len(df_prophage_group) > 1:
        depo_set = set(df_prophage_group[df_prophage_group["Phage"] == prophage]["domain_seq"].values)
        combinations = [depo_set]
        
        for prophage_tmp in df_prophage_group["Phage"].unique():
            if prophage_tmp != prophage:
                tmp_depo_set = set(df_prophage_group[df_prophage_group["Phage"] == prophage_tmp]["domain_seq"].values)
                if tmp_depo_set in combinations:
                    to_exclude.add(prophage_tmp)
                else:
                    to_keep.add(prophage_tmp)
                    combinations.append(tmp_depo_set)
    
    return df_prophage_group, to_exclude, to_keep

def filter_prophages(df_info, dico_prophage_info):
    good_prophages = set()
    excluded_prophages = set()
    for prophage in tqdm(dico_prophage_info.keys()):
        if prophage not in excluded_prophages and prophage not in good_prophages:
            _, excluded_members, kept_members = get_filtered_prophages(prophage, df_info, dico_prophage_info)
            good_prophages.update(kept_members)
            excluded_prophages.update(excluded_members)
    df_info_filtered = df_info[df_info["Phage"].isin(good_prophages)]
    return df_info_filtered[~df_info_filtered["KL_type_LCA"].str.contains("\\|")]

def ultrafilter_prophages(df_info):
    duplicate_prophage = []
    for kltype in df_info["KL_type_LCA"].unique():
        df_kl = df_info[df_info["KL_type_LCA"] == kltype][["Phage", "domain_seq"]]
        set_sets_depo = []
        for _, group in df_kl.groupby("Phage"):
            set_depo = frozenset(group["domain_seq"].values)
            if set_depo in set_sets_depo:
                duplicate_prophage.extend(group["Phage"])
            else:
                set_sets_depo.append(set_depo)
    
    return df_info[~df_info["Phage"].isin(duplicate_prophage)]


def get_mean_MCC(path_data_object):
    try:
        assert os.path.isfile(path_data_object)
        data_object = joblib.load(path_data_object)
        mcc_values = []
        for i in data_object:
            y_test = data_object[i]["test_data"][0].values
            predictions = data_object[i]["test_data"][1]
            mcc = matthews_corrcoef(y_test, predictions)
            mcc_values.append(mcc)
        del data_object 
        mean_mcc = mean(mcc_values)
        return mean_mcc
    except AssertionError:
        raise FileNotFoundError(f"File not found: {path_data_object}")
        


> RF UF

In [ ]:
path_jobs = f"{PATH_WORK}/RF_21122024_models_ultraF"

cluster_values = set([file.split("_")[0] for file in os.listdir(path_jobs)])
KLtypes = set([file.split("_RF_")[1].split(".full_data")[0] for file in os.listdir(path_jobs)])

KLtypes_paths = {kltype : [f"{path_jobs}/{file}" for file in os.listdir(path_jobs) if file.split("_RF_")[1].split(".full_data")[0]==kltype]
                for kltype in KLtypes}

DF_info, dico_prophage_info = load_data()
# UF or not ?
DF_info_lvl_0 = ultrafilter_prophages(filter_prophages(DF_info, dico_prophage_info))
#DF_info_lvl_0 = filter_prophages(DF_info, dico_prophage_info)

dico_prophage_count = Counter(DF_info_lvl_0["KL_type_LCA"])

with open(f"{path_metrics}/SEQ_based.RF__UF.prophages_metrics.tsv", "w") as outfile :
    for KLtype in tqdm(KLtypes_paths) : 
        for path in KLtypes_paths[KLtype] :
            cl_value = path.split("/")[-1].split("_RF")[0]
            mean_mcc = get_mean_MCC(path)
            count_kltype = dico_prophage_count[KLtype]
            outfile.write(f"{KLtype}\t{count_kltype}\t{cl_value}\t{mean_mcc}\n")


> RF :

In [ ]:
path_jobs = f"{PATH_WORK}/RF_1302_models"

cluster_values = set([file.split("_")[0] for file in os.listdir(path_jobs)])
KLtypes = set([file.split("_RF_")[1].split(".full_data")[0] for file in os.listdir(path_jobs)])

KLtypes_paths = {kltype : [f"{path_jobs}/{file}" for file in os.listdir(path_jobs) if file.split("_RF_")[1].split(".full_data")[0]==kltype]
                for kltype in KLtypes}

DF_info, dico_prophage_info = load_data()
# UF or not ?
#DF_info_lvl_0 = ultrafilter_prophages(filter_prophages(DF_info, dico_prophage_info))
DF_info_lvl_0 = filter_prophages(DF_info, dico_prophage_info)

dico_prophage_count = Counter(DF_info_lvl_0["KL_type_LCA"])

with open(f"{path_metrics}/SEQ_based.RF.prophages_metrics.tsv", "w") as outfile :
    for KLtype in tqdm(KLtypes_paths) : 
        for path in KLtypes_paths[KLtype] :
            cl_value = path.split("/")[-1].split("_RF")[0]
            mean_mcc = get_mean_MCC(path)
            count_kltype = dico_prophage_count[KLtype]
            outfile.write(f"{KLtype}\t{count_kltype}\t{cl_value}\t{mean_mcc}\n")


> Lreg reg:

In [ ]:
path_jobs = f"{PATH_WORK}/LR_21122024_models"

cluster_values = set([file.split("_")[0] for file in os.listdir(path_jobs)])
KLtypes = set([file.split("_LogReg_")[1].split(".full_data")[0] for file in os.listdir(path_jobs)])

KLtypes_paths = {kltype : [f"{path_jobs}/{file}" for file in os.listdir(path_jobs) if file.split("_LogReg_")[1].split(".full_data")[0]==kltype]
                for kltype in KLtypes}

DF_info, dico_prophage_info = load_data()
# UF or not ?
#DF_info_lvl_0 = ultrafilter_prophages(filter_prophages(DF_info, dico_prophage_info))
DF_info_lvl_0 = filter_prophages(DF_info, dico_prophage_info)

dico_prophage_count = Counter(DF_info_lvl_0["KL_type_LCA"])

with open(f"{path_metrics}/SEQ_based.LogReg.prophages_metrics.tsv", "w") as outfile :
    for KLtype in tqdm(KLtypes_paths) : 
        for path in KLtypes_paths[KLtype] :
            cl_value = path.split("/")[-1].split("_RF")[0]
            mean_mcc = get_mean_MCC(path)
            count_kltype = dico_prophage_count[KLtype]
            outfile.write(f"{KLtype}\t{count_kltype}\t{cl_value}\t{mean_mcc}\n")

> LogReg UF 

In [ ]:
path_jobs = f"{PATH_WORK}/LR_21122024_models_ultraF"

cluster_values = set([file.split("_")[0] for file in os.listdir(path_jobs)])
KLtypes = set([file.split("_LogReg_")[1].split(".full_data")[0] for file in os.listdir(path_jobs)])

KLtypes_paths = {kltype : [f"{path_jobs}/{file}" for file in os.listdir(path_jobs) if file.split("_LogReg_")[1].split(".full_data")[0]==kltype]
                for kltype in KLtypes}

DF_info, dico_prophage_info = load_data()
# UF or not ?
DF_info_lvl_0 = ultrafilter_prophages(filter_prophages(DF_info, dico_prophage_info))
#DF_info_lvl_0 = filter_prophages(DF_info, dico_prophage_info)

dico_prophage_count = Counter(DF_info_lvl_0["KL_type_LCA"])

with open(f"{path_metrics}/SEQ_based.LogReg__UF.prophages_metrics.tsv", "w") as outfile :
    for KLtype in tqdm(KLtypes_paths) : 
        for path in KLtypes_paths[KLtype] :
            cl_value = path.split("/")[-1].split("_RF")[0]
            mean_mcc = get_mean_MCC(path)
            count_kltype = dico_prophage_count[KLtype]
            outfile.write(f"{KLtype}\t{count_kltype}\t{cl_value}\t{mean_mcc}\n")

***
### Compute the weighted MCC:

In [ ]:
rsync -avzhe ssh \
conchae@garnatxa.srv.cpd:/home/conchae/prediction_depolymerase_tropism/prophage_prediction/depolymerase_decipher/ficheros_28032023/Seqbased_model/metric_files \
/media/concha-eloko/Linux/PPT_clean/ficheros_28032023/review_work/SeqBased_model


In [18]:
path_metrics = "/media/concha-eloko/Linux/PPT_clean/ficheros_28032023/review_work/SeqBased_model/metric_files"

threshold = 5
names_metric_col = ["KL_type", "Count", "file_name","mean_mcc"]
final_weighted_dico = {}

for file in os.listdir(path_metrics):
    metric_df = pd.read_csv(f"{path_metrics}/{file}", sep = "\t", names = names_metric_col)
    metric_eval_df = metric_df[metric_df["Count"] > threshold]
    metric_eval_df["c_value"] = metric_eval_df["file_name"].astype(str).apply(lambda x: x.split("_")[0])
    weighted_mcc_dico = {}
    for cluster in metric_eval_df["c_value"].unique().tolist() :
        mcc_sum = 0
        cl_df = metric_eval_df[metric_eval_df["c_value"] == cluster]
        for _,row in cl_df.iterrows() : 
            mcc_sum = mcc_sum + row["mean_mcc"] * row["Count"]
        weighted_mcc = mcc_sum / (sum(cl_df["Count"]))
        weighted_mcc_dico[cluster] = weighted_mcc
    final_weighted_dico[file] = weighted_mcc_dico



/tmp/ipykernel_142516/2213569197.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  metric_eval_df["c_value"] = metric_eval_df["file_name"].astype(str).apply(lambda x: x.split("_")[0])
/tmp/ipykernel_142516/2213569197.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  metric_eval_df["c_value"] = metric_eval_df["file_name"].astype(str).apply(lambda x: x.split("_")[0])
/tmp/ipykernel_142516/2213569197.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try us

In [19]:
final_weighted_dico

{'SEQ_based.LogReg__UF.prophages_metrics.tsv': {'0.85': 0.3151602636210511,
  '0.9': 0.32933361394227706,
  '0.8': 0.34091186979267774,
  '0.7': 0.36446002765094077,
  '0.95': 0.27893023121060845,
  '0.75': 0.3463067872443405,
  '0.65': 0.36193177410601585,
  '0.975': 0.25498957852463594},
 'SEQ_based.LogReg.prophages_metrics.tsv': {'0.85': 0.3722464784487348,
  '0.9': 0.33694270698415413,
  '0.8': 0.3659430012749742,
  '0.7': 0.36964008602380943,
  '0.95': 0.3319429903096709,
  '0.75': 0.374760037337778,
  '0.65': 0.39023597272237615,
  '0.975': 0.27913217655865474},
 'SEQ_based.RF.prophages_metrics.tsv': {'0.95': 0.7505887697037068,
  '0.9': 0.7665851018307361,
  '0.85': 0.7677152879518742,
  '0.75': 0.7576805724196833,
  '0.975': 0.7326524783190056,
  '0.8': 0.7651531836408012,
  '0.7': 0.7564513341443188,
  '0.65': 0.7666988914668954},
 'SEQ_based.RF__UF.prophages_metrics.tsv': {'0.95': 0.3093359009354269,
  '0.9': 0.3353644615409259,
  '0.85': 0.35113786424314536,
  '0.75': 0.3435

In [ ]:
SEQ_based.LogReg__UF.prophages_metrics.tsv : '0.7': 0.36446002765094077
SEQ_based.LogReg.prophages_metrics.tsv : '0.65': 0.39023597272237615 

SEQ_based.RF.prophages_metrics.tsv : '0.85': 0.7677152879518742
SEQ_based.RF__UF.prophages_metrics.tsv : '0.65': 0.3669266262836219

In [ ]:
import os 

path_work = "/home/conchae/prediction_depolymerase_tropism/prophage_prediction/depolymerase_decipher/ficheros_28032023/Seqbased_model"

dico_models = {"LR_21122024_models" : {"c_value" : "0.65", "destination" : "LogReg"}, 
               #"LR_21122024_models_ultraF" : {"c_value" : "0.7", "destination" : "LogReg_UF"} , 
               #"RF_21122024_models_ultraF" : {"c_value" : "0.65", "destination" : "TropiSEQ_UF"},
               "RF_21122024_models_ultraF" : {"c_value" : "0.7", "destination" : "TropiSEQ_UF_70"},
               #"RF_21122024_models_ultraF" : {"c_value" : "0.85", "destination" : "TropiSEQ_UF_85"},
              }

os.makedirs(f"{path_work}/reviewed_models/TropiSEQ_UF_70", exist_ok=True)

for file in os.listdir(path_work) : 
    if file in dico_models : 
        if file == "RF_21122024_models_ultraF":
            for model in os.listdir(f"{path_work}/{file}") :
                if model.startswith(dico_models[file]["c_value"]) :
                    os.system(f"cp {path_work}/{file}/{model} {path_work}/reviewed_models/{dico_models[file]['destination']}")

In [ ]:
rsync -avzhe ssh \
conchae@garnatxa.srv.cpd:/home/conchae/prediction_depolymerase_tropism/prophage_prediction/depolymerase_decipher/ficheros_28032023/Seqbased_model/reviewed_models \
/media/concha-eloko/Linux/PPT_clean/ficheros_28032023/review_work/SeqBased_model


***
### Write the Metrics file: 

In [3]:
import os 
import pandas as pd
import json 

path_metrics = "/media/concha-eloko/Linux/PPT_clean/ficheros_28032023/review_work/SeqBased_model/metric_files"

df_rf_uf = pd.read_csv(f"{path_metrics}/SEQ_based.RF__UF.prophages_metrics.tsv", sep = "\t", names = ["KL_types", "number_prophages", "c_value", "MCC"])
df_rf_uf = df_rf_uf[df_rf_uf["c_value"] == 0.65]

In [5]:
path_seqbased = "/media/concha-eloko/Linux/PPT_clean/Seqbased_model"

dico_cluster = json.load(open(f"{path_seqbased}/dico_cluster.cdhit__0.65.json"))
dico_cluster_r = {ref_dpo : key_dpo for key_dpo,list_dpo in dico_cluster.items() for ref_dpo in list_dpo}

> DataFrame :

In [11]:
from tqdm import tqdm

path_work = "/media/concha-eloko/Linux/PPT_clean"

df_info = pd.read_csv(f"{path_work}/TropiGATv2.final_df_v2.tsv", sep = "\t" ,  header = 0)
df_prophages = df_info.drop_duplicates(subset = ["Phage"], keep = "first")
dico_prophage_info = {row["Phage"] : {"prophage_strain" : row["prophage_id"] , "ancestor" : row["Infected_ancestor"]} for _,row in df_prophages.iterrows()}

df_info_final = ultrafilter_prophages(filter_prophages(df_info, dico_prophage_info))
df_info_final["dpo_cluster"] = df_info_final["index"].apply(lambda x : dico_cluster_r[x])

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 15981/15981 [00:18<00:00, 883.16it/s]


In [22]:
df_info_final

,Phage,Protein_name,KL_type_LCA,Infected_ancestor,index,Dataset,seq,domain_seq,1,2,...,1273,1274,1275,1276,1277,1278,1279,1280,prophage_id,dpo_cluster
0,GCF_902164905.1__phage1,GCF_902164905.1__phage1__34,KL41,GCF_902164905.1,minibatch__460,minibatch,MPATPQDRLYGLTTSVAVKPPVFISVDYDVARFGEQTITSKTPTDE...,QDRLYGLTTSVAVKPPVFISVDYDVARFGEQTITSKTPTDERTITT...,0.025276,0.053137,...,0.081105,0.012011,0.042917,0.009402,0.093175,-0.080562,0.000897,0.111854,prophage_11309,Dpo_cdhit_73
10,GCF_001701985.1__phage2,GCF_001701985.1__phage2__357,KL30,n4988,minibatch__1084,minibatch,MTVSTQVSRNEYTGNGATTQYDFTFRILDKSHLLVQTMDTSENIVT...,VSTQVSRNEYTGNGATTQYDFTFRILDKSHLLVQTMDTSENIVTLT...,0.004905,0.040896,...,0.087288,0.022292,0.024434,0.025246,0.083449,-0.123537,0.047648,0.061250,prophage_6465,Dpo_cdhit_11
12,GCF_001611095.1__phage5,GCF_001611095.1__phage5__1365,KL30,n49894989,minibatch__1084,minibatch,MTVSTQVSRNEYTGNGATTQYDFTFRILDKSHLLVQTMDTSENIVT...,VSTQVSRNEYTGNGATTQYDFTFRILDKSHLLVQTMDTSENIVTLT...,0.004905,0.040896,...,0.087288,0.022292,0.024434,0.025246,0.083449,-0.123537,0.047648,0.061250,prophage_5,Dpo_cdhit_11
19,GCF_001701935.1__phage4,GCF_001701935.1__phage4__127,KL30,n4988,minibatch__1084,minibatch,MTVSTQVSRNEYTGNGATTQYDFTFRILDKSHLLVQTMDTSENIVT...,VSTQVSRNEYTGNGATTQYDFTFRILDKSHLLVQTMDTSENIVTLT...,0.004905,0.040896,...,0.087288,0.022292,0.024434,0.025246,0.083449,-0.123537,0.047648,0.061250,prophage_5,Dpo_cdhit_11
21,GCF_900622625.1__phage2,GCF_900622625.1__phage2__2892,KL6,GCF_900622625.1,minibatch__1741,minibatch,MAFNPELGSSSPEVLLDNAKRLDELTNGPAATVPDRAGEPLDSWRK...,ELGSSSPEVLLDNAKRLDELTNGPAATVPDRAGEPLDSWRKMQEDN...,-0.003624,-0.032987,...,-0.010726,0.008995,-0.001741,-0.103979,0.119088,-0.124593,0.011745,0.147883,prophage_4098,Dpo_cdhit_117
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21345,GCF_900407145.1__phage13,GCF_900407145.1__phage13__48,KL105,GCF_900407145.1,anubis_return__4247,anubis_return,MAEVPLPTPTQAPVPSTDIRNAVFAGAKLDEEVTGTGEFYTDRLGV...,NIASYDVTWFGAVASDDTATYTAANTVSIQNALNAAEKAGLAAVWF...,-0.002267,0.042625,...,0.112762,0.002557,0.024400,0.066527,0.052364,-0.092847,0.054733,0.077084,prophage_8459,Dpo_cdhit_325
21346,GCF_002186895.1__phage9,GCF_002186895.1__phage9__5,KL57,GCF_002186895.1,anubis_return__4260,anubis_return,MRYRFIALALCLLSGSKVAISAGFDCSLANLSPTEKTICSNEYLSG...,ITDSPWLVKKIFSSDSFEGGINLEGMNVSSILTYQEIKNDLYIYIS...,0.073450,0.046651,...,0.012151,0.003563,-0.022575,0.014130,0.063376,-0.050646,-0.085156,-0.010849,prophage_6002,Dpo_cdhit_643
21347,GCF_004312845.1__phage3,GCF_004312845.1__phage3__38,KL9,GCF_004312845.1,anubis_return__4275,anubis_return,MAILITGKSMTRLPESSSWEEEIELITRSERVAGGLDGPANRPLKS...,DAVIRRDLASDKGTSGVGKLGDKPLVAISYYKSKGQSDQDAVQAAF...,0.032196,0.048856,...,0.084711,0.056063,0.001793,0.073958,0.090169,-0.060105,0.023726,0.086452,prophage_12656,Dpo_cdhit_769
21348,GCF_900172635.1__phage2,GCF_900172635.1__phage2__1608,KL124,GCF_900172635.1,anubis_return__4287,anubis_return,MADLSISVISDQASESNQAGWWHPLDSFQGVEYYGLCKEYGTAGYH...,MADLSISVISDQASESNQAGWWHPLDSFQGVEYYGLCKEYGTAGYH...,-0.011089,-0.005328,...,0.046130,0.012586,-0.021702,-0.023386,0.105700,-0.099147,-0.057367,0.091427,prophage_12780,Dpo_cdhit_268


In [12]:
def get_number_dpocluster(df, kltype) :
    relevant_df = df[df["KL_type_LCA"] == kltype]
    n_cluster = relevant_df["dpo_cluster"].unique()
    return len(n_cluster)

In [14]:
get_number_dpocluster(df_info_final, "KL24")

51

In [21]:
for _, row in df_rf_uf.iterrows() :
    n_clusters = get_number_dpocluster(df_info_final, row["KL_types"])
    ratio = n_clusters / row["number_prophages"]
    print(row["KL_types"], round(row["MCC"], 3), round(ratio, 3), n_clusters , row["number_prophages"], sep = "\t")

KL53	0.526	0.479	23	48
KL24	0.0	0.285	51	179
KL42	0.0	1.0	7	7
KL34	0.0	0.562	9	16
KL107	0.354	0.224	50	223
KL116	0.0	0.515	17	33
KL140	0.341	0.545	12	22
KL82	0.16	0.857	6	7
KL55	0.183	0.632	12	19
KL48	0.0	0.762	16	21
KL30	0.0	0.532	41	77
KL35	0.168	0.682	15	22
KL8	0.0	1.0	16	16
KL142	0.0	0.824	14	17
KL81	0.579	0.51	25	49
KL153	0.403	0.636	14	22
KL28	0.5	0.213	32	150
KL39	0.0	0.68	17	25
KL16	0.0	0.731	19	26
KL155	0.802	0.562	9	16
KL52	0.0	0.6	18	30
KL127	0.411	0.408	20	49
KL143	0.0	0.5	5	10
KL26	0.0	0.733	11	15
KL109	0.461	0.722	13	18
KL146	0.0	1.0	5	5
KL122	0.0	0.654	17	26
KL112	0.0	0.833	15	18
KL13	0.0	0.263	20	76
KL166	0.0	0.9	9	10
KL4	0.0	0.706	12	17
KL114	0.377	0.5	10	20
KL145	0.868	0.396	19	48
KL43	0.284	0.6	33	55
KL63	0.287	0.236	13	55
KL164	0.531	0.778	7	9
KL119	0.0	0.667	6	9
KL103	0.0	0.727	8	11
KL36	0.323	0.438	21	48
KL105	0.286	0.369	24	65
KL25	0.384	0.249	58	233
KL61	0.0	0.75	9	12
KL15	0.0	0.439	47	107
KL74	0.596	0.39	39	100
KL141	0.0	0.556	5	9
KL21	0.379	0.509	27	53
KL126	0